### Import Required Libraries and Set Up Environment Variables

In [59]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize
import re



In [60]:
# Set environment variables from the .env in the local environment
load_dotenv('Howard.env')
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

type(nyt_api_key)
type(tmdb_api_key)

str

### Access the New York Times API

In [61]:
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
#encoded_filter_query = filter_query.replace(' ', '%20').replace('"', '%22').replace(':', '%3A').replace('&', '%26')

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Calculate the number of pages needed to retrieve 200 reviews (assuming 10 articles per page)
num_pages = 20 

# Build URL

#nyt_url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}")
#print("NYT API URL:", nyt_url)


In [62]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(num_pages):
    # Build the NYT URL with the page parameter
    query_url = (
        f"{url}fq={filter_query}&sort={sort}&fl={field_list}"
        f"&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}&page={page}"
    )
       
    # Try and save the reviews to the reviews_list
    try:
        # Make a "GET" request and retrieve the JSON
        response = requests.get(query_url)
        
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            docs = data['response']['docs']
            
            # Loop through the reviews and append each review to the list
            for doc in docs:
                review = {
                    "headline": doc.get("headline", {}).get("main", ""),
                    "web_url": doc.get("web_url", ""),
                    "snippet": doc.get("snippet", ""),
                    "source": doc.get("source", ""),
                    "keywords": [keyword['value'] for keyword in doc.get("keywords", [])],
                    "pub_date": doc.get("pub_date", ""),
                    "byline": doc.get("byline", {}).get("original", ""),
                    "word_count": doc.get("word_count", 0)
                }
                reviews_list.append(review)
            
            # Print the page that was just retrieved
            print(f"Page {page} retrieved successfully.")
        
        # Add a twelve-second interval between queries to stay within API query limits
        time.sleep(12)

    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"Error on page {page}: {e}")
        break

# Print the total number of reviews retrieved
print(f"Total reviews retrieved: {len(reviews_list)}")
            
 

Page 0 retrieved successfully.
Page 1 retrieved successfully.
Page 2 retrieved successfully.
Page 3 retrieved successfully.
Page 4 retrieved successfully.
Page 5 retrieved successfully.
Page 6 retrieved successfully.
Page 7 retrieved successfully.
Page 8 retrieved successfully.
Page 9 retrieved successfully.
Page 10 retrieved successfully.
Page 11 retrieved successfully.
Page 12 retrieved successfully.
Page 13 retrieved successfully.
Page 14 retrieved successfully.
Page 15 retrieved successfully.
Page 16 retrieved successfully.
Page 17 retrieved successfully.
Page 18 retrieved successfully.
Page 19 retrieved successfully.
Total reviews retrieved: 200


In [63]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_five_reviews = reviews_list[:5]

# Convert to JSON formatted string with indentation for readability
nyt_json = json.dumps(first_five_reviews, indent=4)

# Print the JSON string
print(nyt_json)


[
    {
        "headline": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "keywords": [
            "Movies",
            "The Attachment Diaries (Movie)",
            "Diment, Valentin Javier"
        ],
        "pub_date": "2023-05-25T11:00:03+0000",
        "byline": "By Jeannette Catsoulis",
        "word_count": 295
    },
    {
        "headline": "Review: \u2018What\u2019s Love Got to Do With It?\u2019 Probably a Lot",
        "web_url": "https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html",
        "snippet": "Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.",
        "source": "The New York Times",
        

In [64]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = json_normalize(reviews_list)
print(reviews_list_df.head())

reviews_list_df.to_csv('nyt_reviews.csv', index=False)

                                            headline  \
0        ‘The Attachment Diaries’ Review: Love, Sick   
1  Review: ‘What’s Love Got to Do With It?’ Proba...   
2  ‘You Can Live Forever’ Review: Do You Love Me ...   
3  ‘A Tourist’s Guide to Love’ Review: A Wearying...   
4     ‘Other People’s Children’ Review: True Romance   

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook 

In [65]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_list_df = json_normalize(reviews_list)
    
# Define a function to extract the title using regex
def extract_title(headline):
    match = re.search(r'\u2018(.*?)\u2019 Review', headline)
    if match:
        return match.group(1)
    return None

# Apply the function to the DataFrame to create the 'title' column
reviews_list_df['title'] = reviews_list_df['headline'].apply(extract_title)

# Display the DataFrame
print(reviews_list_df.head())

# Save the DataFrame to a CSV file
reviews_list_df.to_csv('nyt_reviews_with_titles.csv', index=False)




                                            headline  \
0        ‘The Attachment Diaries’ Review: Love, Sick   
1  Review: ‘What’s Love Got to Do With It?’ Proba...   
2  ‘You Can Live Forever’ Review: Do You Love Me ...   
3  ‘A Tourist’s Guide to Love’ Review: A Wearying...   
4     ‘Other People’s Children’ Review: True Romance   

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook 

In [66]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    if isinstance(keyword_list, list):
        extracted_keywords = ""
        for item in keyword_list:
            if isinstance(item, dict):
                # Extract 'name' and 'value'
                keyword = f"{item.get['name','unknown']}: {item.get['value', 'unknown']};" 
                # Append the keyword item to the extracted_keywords list
                extracted_keywords += keyword
            return extracted_keywords
    return ""

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df['keywords'] = reviews_list_df['keywords'].apply(extract_keywords)

print(reviews_list_df.head())


                                            headline  \
0        ‘The Attachment Diaries’ Review: Love, Sick   
1  Review: ‘What’s Love Got to Do With It?’ Proba...   
2  ‘You Can Live Forever’ Review: Do You Love Me ...   
3  ‘A Tourist’s Guide to Love’ Review: A Wearying...   
4     ‘Other People’s Children’ Review: True Romance   

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook 

In [67]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_list_df['title'].to_list()
print("Titles List:", title_list)

Titles List: ['The Attachment Diaries', None, 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story', 'Aulcie', 'Love Is Love Is L

### Access The Movie Database API

In [68]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/"
tmdb_api_key = "12a88ae06226457a6ebd254363d8b377"
tmdb_key_string = "&api_key=" + tmdb_api_key


In [69]:
# Create an empty list to store the results
tmdb_movies_list = []
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter  = 0

# Create a set to keep track of processed titles
processed_titles = set()

# Loop through the titles
for title in title_list:

    # Check if the title has already been processed
    if title in processed_titles:
        continue  # Skip processing if title has already been processed
    
    # Add the title to processed_titles set
    processed_titles.add(title)

    # Check if we need to sleep before making a request
    
    if request_counter > 0 and request_counter % 50 == 0:
        print(f"The application is sleeping {request_counter} ...")
        time.sleep(1)  # Pause for 1 second
    
    # Add 1 to the request counter
    request_counter += 1  
   
    # Include a try clause to search for the full movie details.
    try:
        #Construct the search URL
        search_url = f"{url}search/movie?query={title}{tmdb_key_string}"
        
        # Perform a "GET" request for The Movie Database
        response = requests.get(search_url)

        # Construct the search URL
        #search_url = f"{url}search/movie?query={title}&api_key={tmdb_api_key}"

        # Perform a "GET" request for The Movie Database
        #response = requests.get(search_url)
    
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
        if response.status_code == 200:
                movie_search_results = response.json().get('results', [])

                if movie_search_results:
                    # Extract the movie ID of the first result
                    movie_id = movie_search_results[0]['id']

                    # Make a request for a the full movie details
                    movie_details_url = f"{url}movie/{movie_id}?{tmdb_key_string}"
                    #movie_details_url = f"{url}{movie_id}{tmdb_api_key}"
                
                    # Execute "GET" request with url
                    response = requests.get(movie_details_url)

                    if response.status_code == 200:
                        movie_data = response.json()
                    
                        # Extract the genre names into a list
                        genres = [genre['name'] for genre in movie_data.get('genres', [])]

                        # Extract the spoken_languages' English name into a list
                        spoken_languages = [lang['english_name'] for lang in movie_data.get('spoken_languages', [])]

                        # Extract the production_countries' name into a list
                        production_countries = [country['name'] for country in movie_data.get('production_countries', [])]

                        # Add the relevant data to a dictionary and
                        # append it to the tmdb_movies_list list
                        movie_details = {
                            "title": movie_data.get("title", ""),
                            "original_title": movie_data.get("original_title", ""),
                            "budget": movie_data.get("budget", 0),
                            "original_language": movie_data.get("original_language", ""),
                            "homepage": movie_data.get("homepage", ""),
                            "overview": movie_data.get("overview", ""),
                            "popularity": movie_data.get("popularity", 0.0),
                            "runtime": movie_data.get("runtime", 0),
                            "revenue": movie_data.get("revenue", 0),
                            "release_date": movie_data.get("release_date", ""),
                            "vote_average": movie_data.get("vote_average", 0.0),
                            "vote_count": movie_data.get("vote_count", 0),
                            "genres": genres,
                            "spoken_languages": spoken_languages,
                            "production_countries": production_countries
                        }
                        #Append movie details to tdmb_movies_list  
                        tmdb_movies_list.append(movie_details)

                        # Print out the title that was found
                        print(f"Found movie details for: {movie_data.get('title', '')}")
                    else:
                        # Print message if movie details request failed
                        print(f"Failed to retrieve movie details for '{title}'")
                else:
                    #Print message if movie is not found 
                    print(f"Movie '{title}' not found. ")
        else:
            #Print message if search request failed
            print(f"Failed to search for '{title}'")
    except Exception as e:
        print(f"Error occurred: {e}")
        continue  # Continue to next title on error
                        

Found movie details for: The Attachment Diaries
Found movie details for: Red Light Green Light
Found movie details for: You Can Live Forever
Found movie details for: A Tourist's Guide to Love
Found movie details for: Other People's Children
Found movie details for: One True Loves
Found movie details for: The Lost Weekend: A Love Story
Found movie details for: A Thousand and One
Found movie details for: Your Place or Mine
Found movie details for: Love in the Time of Fentanyl
Found movie details for: Pamela, A Love Story
Found movie details for: In from the Side
Found movie details for: After Love
Found movie details for: Alcarràs
Found movie details for: Nelly and Monsieur Arnaud
Found movie details for: Lady Chatterley's Lover
Found movie details for: The Sound of Christmas
Found movie details for: The Inspection
Found movie details for: Bones and All
Found movie details for: My Policeman
Found movie details for: About Fate
Found movie details for: Waiting for Bojangles
Found movie det

In [70]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
# Select the first five elements

first_five_movies = tmdb_movies_list[:5]

# Convert to JSON format with indentation
formatted_json = json.dumps(first_five_movies, indent=4)

# Print the formatted JSON
print(formatted_json)



[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.24,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
  

In [71]:
# Convert the results to a DataFrame

# Convert JSON string back to Python object (list of dictionaries)
movies_data = json.loads(formatted_json)

# Convert the list of dictionaries to a DataFrame
tmdb_df = pd.DataFrame(movies_data)

# Print the first few rows of the DataFrame
print(tmdb_df.head())



                       title             original_title  budget  \
0     The Attachment Diaries                   El apego       0   
1      Red Light Green Light                       None       0   
2       You Can Live Forever       You Can Live Forever       0   
3  A Tourist's Guide to Love  A Tourist's Guide to Love       0   
4    Other People's Children     Les Enfants des autres       0   

  original_language                                           homepage  \
0                es                                                      
1                en                                                      
2                en  https://gooddeedentertainment.com/you-can-live...   
3                en             https://www.netflix.com/title/81424906   
4                fr  https://www.wildbunch.biz/movie/other-peoples-...   

                                            overview  popularity  runtime  \
0  Argentina, 1970s. A desperate young woman goes...       2.240      102  

### Merge and Clean the Data for Export

In [72]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list_df, tmdb_df, on='title')

In [73]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df.head())

                                            headline  \
0        ‘The Attachment Diaries’ Review: Love, Sick   
1  ‘You Can Live Forever’ Review: Do You Love Me ...   

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   

  keywords                  pub_date                    byline  word_count  \
0           2023-05-25T11:00:03+0000    By Jeannette Catsoulis         295   
1           2023-05-04T11:00:08+0000  By Elisabeth Vincentelli         294   

                    title        original_title  ...  \
0  The Attachment Diaries              El apego  ...   
1    You Can Live Forever  You Can Live Forever  ...   

                             

In [74]:
# Drop "byline.person" column
if 'byline.person' in merged_df.columns:
    merged_df.drop('byline.person', axis=1, inplace=True)

In [75]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

In [76]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)